In [1]:
import networkx as nx
import osmnx as ox
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt

ox.__version__

'2.0.1'

In [ ]:
ville = "Vaud, Suisse" # Remplacez par la ville souhaitée, exemple : "Vallorbe, District du Jura-Nord vaudois, Vaud, 1337, Suisse" - noms exacts de la documentation OSM

print("Chargement des graphes...")
G_drive = ox.graph.graph_from_place(ville, network_type="drive")
G_walk = ox.graph.graph_from_place(ville, network_type="walk")

print("Calcul des vitesses pour drive...")
G_drive = ox.routing.add_edge_speeds(G_drive)
G_drive = ox.routing.add_edge_travel_times(G_drive)

print("Application des vitesses personnalisées pour walk...")
G_walk = ox.routing.add_edge_speeds(G_walk)
G_walk = ox.routing.add_edge_travel_times(G_walk)
for u, v, k, data in G_walk.edges(keys=True, data=True):
    data["speed_kph"] = 30  # Vitesse par défaut pour les pompiers sur toutes routes
G_walk_proj = ox.routing.add_edge_travel_times(G_walk)

print("Fusion des graphes...")
G_combined = nx.compose(G_drive, G_walk_proj)

Chargement des graphes...
Calcul des vitesses pour drive...
Application des vitesses personnalisées pour walk...
Fusion des graphes...


In [ ]:
# Liste des types de routes à CONSERVER
print("Supression des types de 'highway' qui ne sont pas dans cette liste; unclassified, track, primary, residential, secondary, service, tertiary.")
types_to_keep = ["unclassified","track", "primary", "residential", "secondary", "service", "services", "tertiary", "motorway", "motorway_link", "primary_link", "secondary_link", "tertiary_link", "trunk", "trunk_link"] 

# Fonction pour vérifier si une arête doit être conservée
def should_keep(highway_value):
    if isinstance(highway_value, list):
        # Vérifier si au moins un des types dans la liste est à conserver
        return any(highway_type in types_to_keep for highway_type in highway_value)
    else:
        # Vérifier si le type unique est à conserver
        return highway_value in types_to_keep

# Identifier les arêtes à supprimer (celles qui ne sont PAS à conserver)
edges_to_remove = []
for u, v, k, data in G_combined.edges(keys=True, data=True):
    if 'highway' in data:
        if not should_keep(data['highway']):
            edges_to_remove.append((u, v, k))
    else:
        # Si l'arête n'a pas d'attribut highway, on la supprime également
        edges_to_remove.append((u, v, k))

# Supprimer les arêtes qui ne sont pas des types à conserver
for edge in edges_to_remove:
    u, v, k = edge
    G_combined.remove_edge(u, v, k)
    
# Mettre à jour les GeoDataFrames avec le graphe modifié
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_combined, nodes=True, edges=True)

# Vérifier les types de highways restants
all_highway_types = set()
for highway_value in gdf_edges['highway']:
    if isinstance(highway_value, list):
        for item in highway_value:
            all_highway_types.add(item)
    else:
        all_highway_types.add(highway_value)

sorted_highway_types = sorted([x for x in all_highway_types if x is not None])
if None in all_highway_types:
    sorted_highway_types.append(None)
    
print("Terminé")


Supression des types de 'highway' qui ne sont pas dans cette liste; unclassified, track, primary, residential, secondary, service, tertiary.
Terminé


In [ ]:
print("5. Conversion en GeoDataFrames...")
gdf_nodes, gdf_edges = ox.convert.graph_to_gdfs(G_combined, nodes=True, edges=True)
print(f"   ➤ {len(gdf_nodes)} nœuds et {len(gdf_edges)} arêtes extraits.")

nom_fichier = "Vaud" # Nom du fichier de sortie, modifiez ici et dans 'calcultateur_itineraire.py' si nécessaire
print("6. Export du graphe en Geopackage...")
ox.io.save_graph_geopackage(G_combined, filepath=nom_fichier + ".gpkg")
print("   ➤ Fichier .gpkg enregistré.")

print("7. Export du graphe en GraphML...")
ox.io.save_graphml(G_combined, filepath=nom_fichier + ".graphml")
print("   ➤ Fichier .graphml enregistré.")

print("✅ Traitement terminé avec succès.")

5. Conversion en GeoDataFrames...
   ➤ 1417 nœuds et 2845 arêtes extraits.
6. Export du graphe en Geopackage...
   ➤ Fichier .gpkg enregistré.
7. Export du graphe en GraphML...
   ➤ Fichier .graphml enregistré.
✅ Traitement terminé avec succès.
